In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
torch.__version__

'1.3.0+cu100'

首先定义一些超参数

In [2]:
BATCH_SIZE = 512
EPOCHS = 20 # 对数据集迭代20次
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu") # 自动判断当前是否支持gpu环境

直接通过DataLoader下载数据并读取

In [3]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('data', train=True, download=True,
                  transform=transforms.Compose([
                      transforms.ToTensor(),
                      transforms.Normalize((0.1307,), (0.3081,))
                  ])),
    batch_size=BATCH_SIZE, shuffle=True
)

0it [00:00, ?it/s]

 99%|█████████▉| 9854976/9912422 [00:15<00:00, 1084252.57it/s]

Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw



0it [00:00, ?it/s]


  0%|          | 0/28881 [00:00<?, ?it/s]
32768it [00:00, 35608.47it/s]                           

0it [00:00, ?it/s]

...


 97%|█████████▋| 1605632/1648877 [00:05<00:00, 441305.91it/s]

0it [00:00, ?it/s]

Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw




8192it [00:00, 14848.23it/s]            

Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw
Processing...
Done!



9920512it [00:29, 1084252.57it/s]                             
1654784it [00:23, 441305.91it/s]                             

获取测试集

In [6]:
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('data', train=False, transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])),
    batch_size=BATCH_SIZE, shuffle=True
)

定义一个卷积神经网络   
网络包含两个卷积层，conv1和conv2，紧接着两个fc层作为输出    
定义输出层为10个维度，根据这个10维度的输出来确定识别的数字结果

In [13]:
class ConvNet(nn.Module):
    def __init__(self):
        super().__init__()
        # 输入数据维度(batch_size, 1, 28, 28)
        # Conv1d参数：输入通道数，输出通道数，卷积核size
        self.conv1 = nn.Conv2d(1, 10, 5)
        # output: [10]
        self.conv2 = nn.Conv2d(10, 20, 3)
        # output: [20]
        # fc层参数: 输入通道数，输出通道数
        self.fc1 = nn.Linear(2000, 500)
        # output: [500]
        self.fc2 = nn.Linear(500, 10)
        # output: [10]
    
    def forward(self, x):
        in_size = x.size(0) 
        # [512, 1, 28, 28]
        out = self.conv1(x)
        # output: [bs, 10, 24, 24]
        out = F.relu(out)
        out = F.max_pool2d(out, 2, 2)
        # output: [bs, 10, 12, 12]
        out = self.conv2(out)
        # output: [bs, 20, 10, 10]
        out = F.relu(out)
        # flatten
        out = out.view(in_size, -1) 
        # output: [bs, 20*10*10=2000]
        out = self.fc1(out)
        # output: [bs, 500]
        out = F.relu(out)
        out = self.fc2(out)
        # output: [bs, 10]
        out = F.log_softmax(out, dim=1) 
        return out


实例化上面定义的神经网络    
使用to方法将网络移动到GPU容器    
优化器使用adam

In [14]:
model = ConvNet().to(DEVICE)
optimizer = optim.Adam(model.parameters())

定义训练脚本

In [20]:
def train(model, device, train_loader, optimiter, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        # 梯度清零
        optimiter.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        # bp梯度
        loss.backward()
        # 更新参数
        optimiter.step() 
        if(batch_idx + 1) % 30 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f})%] \t Loss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

定义测试脚本

In [21]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()
            # 找到概率最大的index
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%) \n'.format(
        test_loss, correct, len(test_loader.dataset), 
        100. * correct / len(test_loader.dataset)))
            

训练

In [22]:
for epoch in range(EPOCHS):
    train(model, DEVICE, train_loader, optimizer, epoch)
    test(model, DEVICE, test_loader)

Train Epoch: 0 [14848/60000 (25)%] 	 Loss: 0.117646
Train Epoch: 0 [30208/60000 (50)%] 	 Loss: 0.109217
Train Epoch: 0 [45568/60000 (75)%] 	 Loss: 0.115753

Test set: Average loss: 0.0765, Accuracy: 9739/10000 (97%) 

Train Epoch: 1 [14848/60000 (25)%] 	 Loss: 0.063976
Train Epoch: 1 [30208/60000 (50)%] 	 Loss: 0.073350
Train Epoch: 1 [45568/60000 (75)%] 	 Loss: 0.151727

Test set: Average loss: 0.0468, Accuracy: 9842/10000 (98%) 

Train Epoch: 2 [14848/60000 (25)%] 	 Loss: 0.054588
Train Epoch: 2 [30208/60000 (50)%] 	 Loss: 0.047075
Train Epoch: 2 [45568/60000 (75)%] 	 Loss: 0.059006

Test set: Average loss: 0.0451, Accuracy: 9857/10000 (99%) 

Train Epoch: 3 [14848/60000 (25)%] 	 Loss: 0.020552
Train Epoch: 3 [30208/60000 (50)%] 	 Loss: 0.020728
Train Epoch: 3 [45568/60000 (75)%] 	 Loss: 0.047147

Test set: Average loss: 0.0370, Accuracy: 9877/10000 (99%) 

Train Epoch: 4 [14848/60000 (25)%] 	 Loss: 0.016404
Train Epoch: 4 [30208/60000 (50)%] 	 Loss: 0.068143
Train Epoch: 4 [45568/60